## BETTER WAY 30 - 속성을 리팩토링하는 대신 `@property`를 고려하자
## Item 30 - Consider `@property` Instead of Refactoring Attributes

내장  `@property` 데코레이터(decorator)를 이용하면 더 간결한 방식으로 인스턴스의 속성에 접근하게 할 수 있다([Better way 29 참고](https://github.com/ExcelsiorCJH/Study/blob/master/Python/Effective-python/Chap04-Metaclasses_and_Attributes/Item29-Use_Plain_Attributes_Instead_of_Get_and_Set_Methods.ipynb))

흔히 사용하는 `@property` 사용법 중 하나는 단순 숫자 속성을 즉석에서 계산하는 방식으로 변경하는 것이다. 호출하는 쪽을 변경하지 않고도 기존에 클래스를 사용한 곳이 새로운 동작을 하게 해주므로 매우 유용한 기법이다.

예를 들어 구멍 난 양동이의 할당량을 일반 파이썬 객체로 구현하려 한다고 해보자. 다음 Bucket 클래스는 남은 할당량과 이 할당량을 이용할 수 있는 기간을 표현한다.

In [1]:
from datetime import datetime, timedelta

class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0
        
    def __repr__(self):
        return 'Bucket(quota=%d)'% self.quota

In [2]:
bucket = Bucket(60)
print(bucket)

Bucket(quota=0)


구멍 난 양동이(leaky bucket) 알고리즘은 양동이를 채울 때마다 할당량이 다음 기간으로 넘어가지 않게 하는 식으로 동작한다.

In [3]:
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

할당량을 소비하는 쪽에서는 매번 사용할 양을 뺄 수 있는지부터 확인해야 한다.

In [4]:
def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

이 클래스를 사용하기 위해 먼저 양동이를 채워보자.

In [5]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


그러고 나서 필요한 만큼 양을 빼보자.

In [6]:
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

Had 99 quota
Bucket(quota=1)


아래의 코드는 이용할 수 있는 양보다 많이 빼려고 해서 진행이 중단되었다. 이 경우 양동이의 할당량은 그대로 남는다.

In [7]:
if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

Not enough for 3 quota
Bucket(quota=1)


이 구현에서 문제는 양동이의 할당량이 어떤 수준에서 시작하는지 모른다는 점이다. 양동이는 0이 될 때까지 진행 기간 동안 할당량이 줄어든다. 0이 되면 deduct가 항상 False를 반환한다. 이때 deduct를 호출하는 쪽이 중단된 이유가 Bucket의 할당량이 소진되어서인지 아니면 처음부터 Bucket에 할당량이 없어서인지 알 수 있다면 좋을 것이다.

문제를 해결하려면 클래스에서 기간 동안 발생한 max_quota와 quota_consumed의 변경을 추적하도록 수정하면 된다.

In [8]:
class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return ('Bucket(max_quota=%d, quota_consumed=%d)'% (self.max_quota, self.quota_consumed))
        
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
        
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # 새 기간의 할당량을 리셋함
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # 새 기간의 할당량을 채움
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # 기간 동안 할당량을 소비함
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [9]:
bucket = Bucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)
 
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')

print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')

print('Still', bucket)

Initial Bucket(max_quota=0, quota_consumed=0)
Filled Bucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now Bucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still Bucket(max_quota=100, quota_consumed=99)


가장 좋은 점은 Bucket.quota를 사용하는 코드는 변경하거나 Bucket 클래스가 변경된 사실을 몰라도 된다는 점이다. Bucket의 새 용법은 제대로 동작하며 max_quota와 quota_consumed에 직접 접근할 수 있다.

### 정리
- 기존의 인스턴스 속성에 새 기능을 부여하려면 `@property`를 사용하자.
- `@property`를 사용하여 점점 나은 데이터 모델로 발전시키자.
- `@property`를 너무 많이 사용한다면 클래스와 이를 호출하는 모든 곳을 리팩토링하는 방안을 고려하자.